# Apache Fineract Integration Demo

This notebook demonstrates how to integrate the AI-Based Transaction Failure Prediction System with Apache Fineract, an open-source core banking platform.

In [ ]:
import sys, os
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd()))
sys.path.insert(0, project_root)

# Import our integration module
from src.integration.fineract_integration import FineractIntegration, integrate_with_fineract_and_predict

# API configuration
PREDICTION_API_URL = "http://localhost:8000"
print(f"Using prediction API at: {PREDICTION_API_URL}")

## Fineract Integration Setup

Set up the connection to Apache Fineract. For this demo, we'll use placeholder values since we don't have a real Fineract instance running.

In [ ]:
# Configuration for Fineract integration
# In a real scenario, these would be real values for your Fineract instance
FINERACT_URL = os.getenv('FINERACT_URL', 'http://localhost:8080/fineract-provider/api/v1/')
FINERACT_USERNAME = os.getenv('FINERACT_USERNAME', 'mifos')
FINERACT_PASSWORD = os.getenv('FINERACT_PASSWORD', 'password')
FINERACT_TENANT = os.getenv('FINERACT_TENANT', 'default')

print(f"Fineract URL: {FINERACT_URL}")
print(f"Fineract Username: {FINERACT_USERNAME}")
print(f"Fineract Tenant: {FINERACT_TENANT}")

# Initialize Fineract integration
fineract = FineractIntegration(
    base_url=FINERACT_URL,
    username=FINERACT_USERNAME,
    password=FINERACT_PASSWORD,
    tenant_id=FINERACT_TENANT
)

print("Fineract integration initialized")

## Authentication with Fineract

Authenticate with the Fineract instance to get an access token.

In [ ]:
# Authenticate with Fineract
try:
    auth_success = fineract.authenticate()
    if auth_success:
        print("Successfully authenticated with Fineract")
    else:
        print("Authentication failed")
        print("Note: For this demo, we'll continue with simulated data")
except Exception as e:
    print(f"Error during authentication: {e}")
    print("Note: For this demo, we'll continue with simulated data")

## Simulate Fineract Transaction Data

Since we don't have a real Fineract instance, we'll simulate transaction data in the format expected by Fineract.

In [ ]:
# Simulate Fineract transaction data
def simulate_fineract_transactions(n=10):
    """Simulate Fineract transaction data in the expected format."""
    transactions = []
    
    for i in range(n):
        # Random transaction date in the last 30 days
        transaction_date = datetime.now() - timedelta(days=np.random.randint(0, 30))
        
        # Random transaction amount
        amount = float(np.random.lognormal(mean=3, sigma=1.5))
        
        # Create a simulated transaction in Fineract format
        transaction = {
            'id': i + 1,
            'date': [transaction_date.year, transaction_date.month, transaction_date.day, transaction_date.hour, transaction_date.minute],
            'amount': amount,
            'runningBalance': {'amount': float(np.random.lognormal(mean=4, sigma=1))},
            'type': {
                'id': 1,
                'code': 'account.transfers.type.debit' if np.random.random() > 0.5 else 'account.transfers.type.credit',
                'value': 'Debit' if np.random.random() > 0.5 else 'Credit'
            },
            'submittedOnDate': [transaction_date.year, transaction_date.month, transaction_date.day],
            'isReversed': False,
            'officeId': 1,
            'officeName': 'Head Office',
            'clientName': f'Client {i+1}',
            'clientId': i + 1
        }
        transactions.append(transaction)
    
    return transactions

# Generate simulated transactions
simulated_transactions = simulate_fineract_transactions(10)
print(f"Simulated {len(simulated_transactions)} transactions in Fineract format")

# Display first transaction
print("\nFirst transaction sample:")
for key, value in simulated_transactions[0].items():
    print(f"  {key}: {value}")

## Transform Fineract Transactions

Transform the simulated Fineract transactions into the format expected by our prediction model.

In [ ]:
# Transform transactions to model input format
transformed_transactions = []

for transaction in simulated_transactions:
    transformed = fineract.transform_fineract_transaction(transaction)
    transformed_transactions.append(transformed)

print(f"Transformed {len(transformed_transactions)} transactions to model input format")

# Display first transformed transaction
print("\nFirst transformed transaction:")
for key, value in transformed_transactions[0].items():
    print(f"  {key}: {value}")

## Make Predictions on Transformed Transactions

Use the prediction API to predict failure probabilities for the transformed transactions.

In [ ]:
# Check if the prediction API is running
try:
    response = requests.get(f"{PREDICTION_API_URL}/health")
    if response.status_code == 200:
        print(f"Prediction API is running: {response.json()}")
    else:
        print(f"Prediction API not responding. Status: {response.status_code}")
        print("Please start the API with: uvicorn src.api.main:app --host 0.0.0.0 --port 8000")
except requests.exceptions.ConnectionError:
    print("Prediction API is not running. Please start it with: uvicorn src.api.main:app --host 0.0.0.0 --port 8000")

# Make predictions for the first 5 transactions
prediction_results = []
for i, transformed in enumerate(transformed_transactions[:5]):
    try:
        response = requests.post(
            f"{PREDICTION_API_URL}/predict",
            json=transformed,
            headers={'Content-Type': 'application/json'}
        )
        
        if response.status_code == 200:
            result = response.json()
            prediction_results.append({
                'original_transaction': simulated_transactions[i],
                'transformed_input': transformed,
                'prediction': result
            })
            print(f"Transaction {i+1}: Prob={result['failure_probability']:.3f}, "
                  f"Pred={'Failure' if result['prediction'] == 1 else 'Success'}")
        else:
            print(f"API request failed for transaction {i+1}: {response.status_code}")
            
    except requests.exceptions.ConnectionError:
        print(f"Cannot connect to API for transaction {i+1}. API might not be running.")
        break
    except Exception as e:
        print(f"Error making prediction for transaction {i+1}: {e}")

print(f"\nMade predictions for {len(prediction_results)} transactions")

## Batch Prediction on Fineract Transactions

Perform batch predictions on multiple transactions at once.

In [ ]:
# Prepare batch request
batch_transactions = []
for transformed in transformed_transactions[:5]:
    # Remove Fineract-specific fields for the batch request
    model_input = {k: v for k, v in transformed.items() 
                   if k not in ['fineract_transaction_id', 'fineract_transaction_type', 'fineract_transaction_date']}
    batch_transactions.append(model_input)

batch_request = {"transactions": batch_transactions}

# Make batch prediction
try:
    response = requests.post(
        f"{PREDICTION_API_URL}/predict_batch",
        json=batch_request,
        headers={'Content-Type': 'application/json'}
    )
    
    if response.status_code == 200:
        batch_results = response.json()
        print(f"Batch prediction results for {len(batch_results)} transactions:")
        
        for i, result in enumerate(batch_results):
            print(f"  Transaction {i+1}: Prob={result['failure_probability']:.3f}, "
                  f"Pred={'Failure' if result['prediction'] == 1 else 'Success'}, "
                  f"Reason: {result['failure_reason']}")
    else:
        print(f"Batch prediction failed: {response.status_code} - {response.text}")
        
except requests.exceptions.ConnectionError:
    print("Cannot connect to API for batch prediction. API might not be running.")
except Exception as e:
    print(f"Error in batch prediction: {e}")

## Integration Summary

Summarize the integration process and results.

In [ ]:
# Create a summary DataFrame
if prediction_results:
    summary_data = []
    for result in prediction_results:
        summary_data.append({
            'Transaction ID': result['original_transaction']['id'],
            'Amount': result['transformed_input']['transaction_amount'],
            'Account Balance': result['transformed_input']['account_balance'],
            'Failure Probability': result['prediction']['failure_probability'],
            'Prediction': 'Failure' if result['prediction']['prediction'] == 1 else 'Success',
            'Failure Reason': result['prediction']['failure_reason']
        })
    
    summary_df = pd.DataFrame(summary_data)
    print("Prediction Summary:")
    print(summary_df)
    
    # Statistics
    failure_count = len(summary_df[summary_df['Prediction'] == 'Failure'])
    success_count = len(summary_df[summary_df['Prediction'] == 'Success'])
    avg_failure_prob = summary_df['Failure Probability'].mean()
    
    print(f"\nStatistics:")
    print(f"  Total transactions: {len(summary_df)}")
    print(f"  Predicted failures: {failure_count} ({failure_count/len(summary_df)*100:.1f}%)")
    print(f"  Predicted successes: {success_count} ({success_count/len(summary_df)*100:.1f}%)")
    print(f"  Average failure probability: {avg_failure_prob:.3f}")
else:
    print("No predictions were made. Please ensure the prediction API is running.")

## Real Integration Process

In a real-world scenario, the integration would work as follows:

In [ ]:
print("Real Integration Process:")
print("1. Set up Apache Fineract server")
print("2. Configure authentication credentials")
print("3. Authenticate with Fineract API")
print("4. Retrieve transaction data from Fineract")
print("5. Transform Fineract data to model input format")
print("6. Send transformed data to prediction API")
print("7. Receive and process prediction results")
print("8. Store or report predictions as needed")

print("\nFor a real implementation, you would:")
print("- Run Apache Fineract server")
print("- Use real authentication credentials")
print("- Retrieve actual transaction data from Fineract")
print("- Implement proper error handling and logging")
print("- Set up monitoring for the integration")

## Summary

We have demonstrated the integration process between the AI-Based Transaction Failure Prediction System and Apache Fineract:
1. Set up the Fineract integration class
2. Simulated Fineract transaction data
3. Transformed the data to the format expected by our prediction model
4. Made predictions using the prediction API
5. Performed batch predictions
6. Summarized the results

In a real implementation, this integration would connect to an actual Apache Fineract instance to retrieve real transaction data and apply AI-based failure predictions.